In [162]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [163]:
x = torch.rand(3)

In [164]:
x

tensor([0.1816, 0.9974, 0.7702])

In [173]:
p = torch.FloatTensor([[0, -1, 1, 1, 1],[1, -2, 0, 2, 1], [0, -1, 0, 2, 1]]).t()
p

tensor([[ 0.,  1.,  0.],
        [-1., -2., -1.],
        [ 1.,  0.,  0.],
        [ 1.,  2.,  2.],
        [ 1.,  1.,  1.]])

In [174]:
p[0,1]

tensor(1.)

In [175]:
class PosyNet(nn.Module):

    def __init__(self):
        super(PosyNet, self).__init__()
        
        # input: 5-vector (state of rocket: x, z, vx, vz, m)
        # output: 2-vector (thrust: mag, direction)
        
        # hidden: 3-posynomial layer
        
        #self.powers = nn.Parameter(torch.rand(5,3))
        self.powers = torch.FloatTensor([[0, -1, 1, 1, 1],[1, -2, 0, 2, 1], [0, -1, 0, 2, 1]]).t()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(3, 2)  

    def forward(self, x):
        
        p = self.powers
        
        #compute posynomial layer
        posy = torch.ones(3)
        #for j in range(3):
            #for i in range(5):
                #posy[j] = posy[j]*x[i]**self.powers[i,j]
        posy[0] = x[0]**p[0,0] * x[1]**p[1,0] * x[2]**p[2,0] * x[3]**p[3,0] * x[4]**p[4,0]
        posy[1] = x[0]**p[0,1] * x[1]**p[1,1] * x[2]**p[2,1] * x[3]**p[3,1] * x[4]**p[4,1]
        posy[2] = x[0]**p[0,2] * x[1]**p[1,2] * x[2]**p[2,2] * x[3]**p[3,2] * x[4]**p[4,2]
        
        #linear layer
        
        out = self.fc1(posy)
        
        return out
    
net = PosyNet()
print(net)

PosyNet(
  (fc1): Linear(in_features=3, out_features=2, bias=True)
)


In [176]:
list(net.parameters())

[Parameter containing:
 tensor([[ 0.4140,  0.3093, -0.5531],
         [-0.3995, -0.0851, -0.1794]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1026, -0.0990], requires_grad=True)]

In [177]:
x = torch.rand(5, requires_grad=True)
x

tensor([0.1386, 0.6472, 0.9969, 0.6178, 0.4837], requires_grad=True)

In [178]:
net(x)

tensor([ 0.1543, -0.3393], grad_fn=<AddBackward0>)

In [179]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [180]:
def compute_actual(state):
    
    x, z, vx, vz, m = state
    
    tgo = -3*z/vz
    
    ax = -6*vx/tgo - 12*x/tgo**2
    
    az = -6*vz/tgo - 12*z/tgo**2 + 1.6
    
    return torch.stack([m*ax, m*az])
    

In [184]:
running_loss = 0
for epoch in range(10000):
    state = torch.rand(5, requires_grad=True)
    actual = compute_actual(state).detach()
    
    optimizer.zero_grad()
    net_prediction = net(input)
    
    loss = criterion(net_prediction, actual)
    
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    
    if epoch%1000==0: 
        print(running_loss)
        running_loss = 0
    

24342.814453125
155103178.22853047
21181429.639840446
1704755168.93831
33829142.13782205
4155243834.6434817
9808790026.693506
59563894051.741135
1365543349.2600818
9.744878884550597e+16


In [185]:
list(net.parameters())

[Parameter containing:
 tensor([[-2.0723, -5.8458, -6.3098],
         [-0.3562,  0.0219, -0.0799]], requires_grad=True),
 Parameter containing:
 tensor([-97.5541,   1.6039], requires_grad=True)]

In [87]:
x

tensor(0.3830, grad_fn=<SelectBackward>)

In [88]:
tgo = -3*z/vz
tgo

tensor(-3.2347, grad_fn=<DivBackward0>)

In [89]:
ax = -6*vx/tgo - 12*x/tgo**2
ax

tensor(-0.0818, grad_fn=<SubBackward0>)

In [90]:
az = -6*vz/tgo - 12*z/tgo**2 + 1.6
az

tensor(1.9743, grad_fn=<AddBackward0>)

In [92]:
torch.stack([ax, az])

tensor([-0.0818,  1.9743], grad_fn=<StackBackward>)

In [96]:
compute_actual(state)

tensor([0.0152, 1.6001], grad_fn=<StackBackward>)

In [98]:
optimizer.zero_grad()


input = torch.rand(5, requires_grad = True)

with torch.no_grad():
    actual = compute_actual(input)


In [99]:
actual

tensor([-0.3630,  2.0294])

In [104]:
net_prediction = net(input)
net_prediction

tensor([-0.0840,  0.0650], grad_fn=<AddBackward0>)

NameError: name 'net_prediction' is not defined

In [106]:
loss = criterion(net_prediction, actual)
loss

tensor(1.9683, grad_fn=<MseLossBackward>)

In [113]:
with torch.autograd.set_detect_anomaly(True):
    loss.backward()

../torch/csrc/autograd/python_anomaly_mode.cpp:40: UserWarning: No forward pass information available. Enable detect anomaly during forward pass for more information.


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [ ]:
print('started training')
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')